## 迭代器

可以直接作用于for循环的对象统称为可迭代对象：`Iterable`

一类是集合数据类型，如list、tuple、dict、set、str等；

一类是generator，包括生成器和带yield的generator function。

可以使用`isinstance()`判断一个对象是否是`Iterable`对象：

In [2]:
from collections.abc import Iterable
isinstance([], Iterable)
isinstance((x for x in range(10)), Iterable)

True

生成器不但可以作用于for循环，还可以被`next()`函数不断调用并返回下一个值，直到最后抛出`StopIteration`错误表示无法继续返回下一个值了。

可以被`next()`函数调用并不断返回下一个值的对象称为迭代器：`Iterator`

可以使用`isinstance()`判断一个对象是否是`Iterator`对象：

In [6]:
from collections.abc import Iterator
isinstance((x for x in range(10)), Iterator)
isinstance([], Iterator)

False

生成器都是`Iterator`对象，但list、dict、str虽然是`Iterable`，却不是`Iterator`。

把list、dict、str等Iterable变成Iterator可以使用`iter()`函数

In [7]:
isinstance(iter('abc'), Iterator)

True

`Iterator`甚至可以表示一个无限大的数据流，例如全体自然数。而使用list是永远不可能存储全体自然数的。

### 小结
凡是可作用于`for`循环的对象都是`Iterable`类型；

凡是可作用于`next()`函数的对象都是`Iterator`类型，它们表示一个惰性计算的序列；

集合数据类型如list、dict、str等是Iterable但不是Iterator，可以通过`ter()`函数获得一个Iterator对象。

Python的for循环本质上就是通过不断调用`next()`函数实现

## 高阶函数

### yield
 - yield在函数中的功能类似于return，不同的是yield每次返回结果之后函数并没有退出，而是 每次遇到yield关键字后返回相应结果，并保留函数当前的运行状态，等待下一次的调用

 - 如果 一个函数需要多次循环执行一个动作，并且每次执行的结果都是需要的，这种场景很适合使用yield实现。

包含yield的函数成为一个生成器，生成器同时也是一个迭代器，支持通过next方法获取下一个值



In [23]:
def foo():
    while True:
        res = yield 4
        print(res)
        
g = foo()
print(next(g))
print('*'*20)
print(next(g))

4
********************
None
4
********************
None
4


1.程序开始执行以后，因为foo函数中有yield关键字，所以foo函数并不会真的执行，而是先得到一个生成器g(相当于一个对象)

2.直到调用next方法，foo函数正式开始执行，先执行foo函数中的print方法，然后进入while循环

3.程序遇到yield关键字，然后把yield想想成return,return了一个4之后，程序停止，并没有执行赋值给res操作，此时next(g)语句执行完成，所以输出的前两行（第一个是while上面的print的结果,第二个是return出的结果）是执行print(next(g))的结果，

4.程序执行print("*"*20)，输出20个*

5.又开始执行下面的print(next(g)),这个时候和上面那个差不多，不过不同的是，这个时候是从刚才那个next程序停止的地方开始执行的，也就是要执行res的赋值操作，这时候要注意，这个时候赋值操作的右边是没有值的（因为刚才那个是return出去了，并没有给赋值操作的左边传参数），所以这个时候res赋值是None,所以接着下面的输出就是res:None,

6.程序会继续在while里执行，又一次碰到yield,这个时候同样return 出4，然后程序停止，print函数输出的4就是这次return出的4.

 

yield和return的关系和区别:带yield的函数是一个生成器，而不是一个函数了，这个生成器有一个函数就是next函数，next就相当于“下一步”生成哪个数，这一次的next开始的地方是接着上一次的next停止的地方执行的，所以调用next的时候，生成器并不会从foo函数的开始执行，只是接着上一步停止的地方开始，然后遇到yield后，return出要生成的数，此步就结束。


### map/reduce

#### map
`map()`函数接收两个参数，一个是函数，一个是`Iterable`，`map`将传入的函数依次作用到序列的每个元素，并把结果作为新的`Iterator`返回。

比如我们有一个函数 f(x) = x^2, 现在需要将这个函数作用在一个list=[1,2,3,4,5,6,7], 可以使用`map`实现:

In [9]:
def f(x):
    return x*x
r = map(f, [1,2,3,4,5,6,7])
print(list(r))

[1, 4, 9, 16, 25, 36, 49]


`map`传入的地哦一个参数是f， 即函数本身。由于结果r 是一个`Iterator`，`Iterator` 是惰性序列。因此通过`list()`函数可以让他把整个序列都计算出来并返回一个list.

In [11]:
L = []
for n in range(1,10):
    L.append(f(n))
print(L)

[1, 4, 9, 16, 25, 36, 49, 64, 81]


把list所有数字转化为字符串

In [ ]:
list(map(str, [1,2,23,4,4,4,5]))

#### Reduce

`reduce`把一个函数作用在一个序列`[x1, x2, x3, ...]`上，这个函数必须接收两个参数，`reduce`把结果继续和序列的下一个元素做累积计算，其效果就是：
```
reduce(f,[x1,x2,x3,x4]) = f(f(f(x1,x2,),x3), x4)
```
以下实例是对一个序列求和

In [13]:
from functools import reduce
def add(x, y):
    return x+y

output = reduce(add, [1,3,5,7])
print(output)

16


把字符串转化为数字：

In [17]:
from functools import reduce

DIGITS = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}

def str2int(s):
    def fn(x,y):
        return x*10 + y
    def char2num(s):
        return DIGITS[s]
    return reduce(fn,map(char2num, s))

### filter

和`map()`类似，`filter()`也接收一个函数和一个序列。和`map()`不同的是，`filter()`把传入的函数依次作用于每个元素，然后根据返回值是True还是False决定保留还是丢弃该元素。

In [19]:
def is_odd(x):
    return x%2 == 1

filtered = filter(is_odd, [1,2,3,4,5,6])
print(list(filtered))

[1, 3, 5]


`filter()`函数返回的是一个`Iterator`，也就是一个惰性序列，所以要强迫`filter()`完成计算结果，需要用list()函数获得所有结果并返回list。

#### 用filter求素数

构造一个从3开始的序列

In [32]:
def _odd_iter():
    n  = 1
    while True:
        n  = n + 2
        yield n

这是一个生成器，并且是一个无限序列， 然后定义一个筛选函数：

In [36]:
def _not_divisible(n):
    return lambda x: x%n >0

True

最后定义一个生成器，不断返回下一个素数：

In [28]:
def primes():
    yield 2
    it = _odd_iter()
    while True:
        n = next(it)
        yield n
        it = filter(_not_divisible(n), it)

这个生成器先返回第一个素数2，然后，利用`filter()`不断产生筛选后的新的序列。

由于`primes()`也是一个无限序列，所以调用时需要设置一个退出循环的条件：

In [33]:
# 打印1000以内的素数:
for n in primes():
    if n < 1000:
        print(n)
    else:
        break

2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97
101
103
107
109
113
127
131
137
139
149
151
157
163
167
173
179
181
191
193
197
199
211
223
227
229
233
239
241
251
257
263
269
271
277
281
283
293
307
311
313
317
331
337
347
349
353
359
367
373
379
383
389
397
401
409
419
421
431
433
439
443
449
457
461
463
467
479
487
491
499
503
509
521
523
541
547
557
563
569
571
577
587
593
599
601
607
613
617
619
631
641
643
647
653
659
661
673
677
683
691
701
709
719
727
733
739
743
751
757
761
769
773
787
797
809
811
821
823
827
829
839
853
857
859
863
877
881
883
887
907
911
919
929
937
941
947
953
967
971
977
983
991
997


### 返回函数

#### 函数作为返回值

不需要立刻求和，而是在后面的代码中，根据需要再计算 ---- 返回求和的函数：



In [38]:
def lazy_sum(*args):
    def sum():
        ax = 0
        for n in args:
            ax = ax + n
        return ax
    return sum

当我们调用lazy_sum()时，返回的并不是求和结果，而是求和函数

In [39]:
f = lazy_sum(1,3,5,7,9)
print(f)

<function lazy_sum.<locals>.sum at 0x7f1c92e37550>


In [40]:
f()

25

#### 闭包

注意到返回的函数在其定义内部引用了局部变量args，所以，当一个函数返回了一个函数后，其内部的局部变量还被新函数引用，所以，闭包用起来简单，实现起来可不容易。